## Analysis of geographical information on the Repertoire

The Repertoir includes two lists of Jesuits residences in China.

One reports to 1644 [Planche: Carte des Chrétientés Chinoises de la fin des Ming (1644)](../sources/dehergne-locations-1644.cli)
and the second to 1701  [VII. Carte des résidences de Chine en l'année 1701](../sources/dehergne-locations-1701.cli).


In [3]:

from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook(
    kleio_version='12.6.576')
tlnb.print_info(show_token=True)

Timelink version: 1.1.14
Project name: dehergne-locations
Project home: /Users/jrc/mhk-home/sources/dehergne-locations
Database type: sqlite
Database name: dehergne_locations
Kleio image: timelinkserver/kleio-server
Kleio server token: jLZtIeWrMF17H5GoLXz1n4Sf1Gt6EKYf
Kleio server URL: http://127.0.0.1:8092
Kleio server home: /Users/jrc/mhk-home/sources/dehergne-locations
Kleio server container: dreamy_ramanujan
Kleio version requested: 12.6.576
Kleio server version: 12.6.576 (2024-09-09 17:57:04)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne-locations/database/sqlite
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne-locations, project_home=/Users/jrc/mhk-home/sources/dehergne-locations, db_type=sqlite, db_name=dehergne_locations, kleio_image=timelinkserver/kleio-server, kleio_version=12.6.576, postgres_image=postgres, postgres_version=latest)


## List of residences in 1644

In [37]:
from sqlalchemy import select
from timelink.kleio.utilities import get_extra_info

def get_wikidata_id(obs, if_missing=''):
    """ Check the obs field for wikidata links

    Returns a tuple of the cleaned comment and the wikidata id"""
    clean_comment, extra_info = get_extra_info(obs)
    name_comment = extra_info.get('name', {}).get('comment','')
    name_original = extra_info.get('name', {}).get('original','')

    if '@wikidata:' in name_comment:
        wikidata = name_comment.split('@wikidata:')[1].split('}')[0]
    elif '@wikidata:' in name_original:
        wikidata = name_original.split('@wikidata:')[1].split('}')[0]
    else:
        wikidata = if_missing
    return clean_comment,wikidata


geo1, geo2, geo3 = tlnb.db.get_model(['geo1','geo2','geo3'])
stmt = select(geo1).where(geo1.inside == 'deh-chre-1644')

with tlnb.db.session() as session:
    result = session.execute(stmt).fetchall()
    for province, in result:
        comment, wikidata = get_wikidata_id(province.obs, if_missing='No wikidata')
        print(province.name, wikidata, comment)
        fous = session.execute(select(geo2).where(geo2.inside == province.id)).fetchall()
        for fou, in fous:
            comment, wikidata = get_wikidata_id(fou.obs, if_missing='No wikidata')
            print(' ', fou.name, comment, wikidata)
            geo3s = session.execute(select(geo3).where(geo3.inside == fou.id)).fetchall()
            for tcheou_hien, in geo3s:
                if tcheou_hien.name == 'Hungtang':
                    pass
                comment, wikidata = get_wikidata_id(tcheou_hien.obs, if_missing='No wikidata')
                print('   ', tcheou_hien.name, comment, wikidata )



Chekiang Q16967 
  Hangchou  Q4970
    Fuyang  Q1011103
    Jenho no wikidata. No wikidata
  Chüchow in the Chinese translation it is recognized as “遂州”, which is wrong, both phonetically and geographically. In Dehergne(1957), it is noted as "衢州". Q58235, in the Chinese translation it is recognized as “遂州”, which is wrong, both phonetically and geographically. In Dehergne(1957), it is noted as "衢州".
  Huchow  Q42664
    Tehtsing  Q1191987"
  Kashing  Q58178
    Kashan """
               The place name in the book Dehergne(1973) is "Kaoshan", and in the Chinese translation it is recognized as “高山”. But this place cannot be found. According to Dehergne(1957), it probably shoud be "Kashan 嘉善", which means in the 1973 book, this place is wrongly spelled by mistake.""" Q1361347
    Tangsi  Q10931032, in the Chinese translation it is recognized as “塘拪”
    Tsungteh  Q10270889, Historical county name, located in the present Chongfu 崇福镇
    Tungsiang  Q1204548
  Kinhwa  Q58210
    Lanchi  Q102